In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from trade_models.n1_torch1_dataset import Dataset, to_device
from trade_models.n1_torch1_model import ResNet28
from time import time
from tqdm.notebook import tqdm
import json

# Model Saving/Loading Methods

In [2]:
def load_model_only(config):
    path = config.get('model_path', '')
    f = f"{path}/{config['model_identifier']}.pth"
    print("Loading existing model")
    checkpoint = torch.load(f)
    net = checkpoint['net']
    mean_losses = checkpoint['mean_losses']
    return net, mean_losses

def init_h1_weights(m):
    if type(m) == nn.Linear:
        nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        m.bias.data.fill_(0.01)
        
def load_model_with_config(config, X_train=None, model_width=None, force_train=False):
    # a bit hacky, but in the training phase, we never load and use the minmax scalers
    # just putting it here for when we want to load the model elsewhere THEN revert scaling
    # probably better to have the scalers saved separately....
    device = config.get('device','cpu')
    pyt_device = torch.device(device)
    
    path = config.get('model_path', '')
    f = f"{path}/{config['model_identifier']}.pth"
    
    if os.path.exists(f) and not force_train:
        print("Loading existing model")
        checkpoint = torch.load(f)
        net = checkpoint['net']
        next_epoch = checkpoint['next_epoch']
        loss_func = checkpoint['loss_func']
        optimizer = checkpoint['optimizer']
        scaler = checkpoint['scaler']
        mean_losses = checkpoint['mean_losses']
    else:
        if X_train is None:
            raise Exception('Cannot create model without X_train')
        print("New model created")
        net = ResNet28(input_size=model_width[0], output_size=model_width[1], width=config['model_width'])
        net.apply(init_h1_weights)
        pos_weight = to_device(torch.tensor([config['loss'].get('pos_weight',1)]), pyt_device) # increase/decrease precision
        loss_func = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        opt_config=config['optimizer']
        #optimizer = torch.optim.SGD(net.parameters(), lr=opt_config['lr'], momentum=opt_config['momentum'])
        optimizer = torch.optim.AdamW(net.parameters(), lr=opt_config['lr'], weight_decay=opt_config['weight_decay'])
        scaler = CustomScaler2().fit(X_train)
        print (scaler)
        mean_losses = []
        next_epoch = 0
        save_model_with_config(config, net=net, loss_func=loss_func, optimizer=optimizer,
                               scaler=scaler,
                               mean_losses=mean_losses, next_epoch=next_epoch,
                              )
        # blank scaler when creating new model
    return net, loss_func, optimizer, scaler, mean_losses, next_epoch

def save_model_with_config(config, **kwargs):
    path = config.get('model_path', '')

    f = f"{path}/{config['model_identifier']}.pth"
    torch.save(kwargs, f)

# Custom Scaler

In [3]:
from sklearn.preprocessing import StandardScaler
import re

class CustomScaler2():
    stdcols = ('number_of_trades','volume','quote_asset_volume',
               'taker_buy_base_asset_volume','taker_buy_quote_asset_volume',
               )
    
    def __init__(self):
        self.standard_scaler = StandardScaler()
        self.tostd = []
        self.is_fit = False
        
    def fit(self, X):
        for c in self.stdcols:
            for v in X.columns:
                if v.startswith(c):
                    self.tostd.append(v)
                
        if len(self.tostd) > 0:
            self.standard_scaler.fit(X[self.tostd])
            
        self.is_fit = True
        return self
        
    def transform(self, X_in):
        if self.is_fit == True:
            X = X_in.copy()
            open_price = X['open'].copy()
            stddev = (((X['close'] - X['open'])**2 + \
                       (X['high'] - X['open'])**2 + \
                       (X['low'] - X['open'])**2) / 3)**0.5
            stddev = stddev.apply(lambda x: 1 if x==0 else x)
            
            rsi_values = X['rsi'].copy()
            atr_values = X['atr'].copy()
            atr_values = atr_values.apply(lambda x: 1 if x==0 else x)
            
            for c in X.columns:
                if c in ['open','high','low','close'] or \
                   re.match('open_[0-9]+', c) or \
                   re.match('high_[0-9]+', c) or \
                   re.match('low_[0-9]+', c) or \
                   re.match('close_[0-9]+', c) or \
                   re.match('sup[0-9]+', c) or \
                   re.match('res[0-9]+', c) or \
                   re.match('ma[0-9]+', c):
                    X[c] = (X[c]-open_price)/stddev
                elif c.startswith('atr_diff'):
                    X[c] = X[c]/atr_values
                elif re.match('atr$|atr_[0-9]+|atr_ma[0-9]+',c):
                    X[c] = X[c]-atr_values/atr_values
                elif c.startswith('rsi_diff'):
                    X[c] = X[c]/rsi_values
                elif re.match('rsi$|rsi_[0-9]+|rsi_ma[0-9]+',c):
                    X[c] = (X[c]-50)/20 # thus the 30/70 thresholds will become -1/1
            
            if 'dow' in X.columns:
                X['dow'] = X['dow'] / 6
                
            if len(self.tostd) > 0:
                X[self.tostd] = self.standard_scaler.transform(X[self.tostd])
            return X
        else:
            raise Exception('CustomScaler not yet fit')
            
    def fit_transform(self, X_in, y=None):
        self.fit(X_in)
        y=self.transform(X_in)
        return y

In [4]:
# pd.set_option('max_columns',500)
# scaler = CustomScaler1()
# X_new = scaler.fit_transform(X_train)
# X_new.head()

# Model Training/Predicting Methods

In [5]:
def train_model(X_train, y_train, X_test, y_test, configurations, force_train=False):

    path = configurations.get('model_path', None)
    torch.manual_seed(configurations.get('random_seed',0))
    device = configurations.get('device','cpu')
    pyt_device = torch.device(device)

    model_width = (X_train.shape[1], y_train.shape[1])
    net, loss_func, optimizer, scaler, mean_losses, next_epoch, = load_model_with_config(configurations,
                                                                                         X_train,
                                                                                         model_width,
                                                                                         force_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    training_set = Dataset(X_train, y_train)
    training_generator = torch.utils.data.DataLoader(training_set, **configurations['train_params'])
    testing_set = Dataset(X_test, y_test)
    testing_generator = torch.utils.data.DataLoader(testing_set, **configurations['test_params'])
    
    to_device(net, pyt_device)
    net.train()
    print(net)

    if next_epoch == configurations['max_epochs']:
        print("Model finished training. To retrain set force_train = True ")
        net.eval()
        return net, mean_losses

    epbar = tqdm(range(next_epoch, configurations['max_epochs']))
    for epoch in epbar:
        epbar.set_description(f"Epoch {epoch+1}")

        running_eloss = 0
        running_vloss = 0

        ipbar = tqdm(training_generator, leave=False)
        ipbar.set_description(f"Training")

        for i, (x, y) in enumerate(ipbar):
            x = to_device(x, pyt_device)
            y = to_device(y, pyt_device)

            optimizer.zero_grad()
            prediction = net(x)     # input x and predict based on x
            loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients
            running_eloss += loss.item()

        net.eval()
        mean_vlosses = 0
        predicted_true = 0
        target_true = 0
        correct_true = 0
        
        if configurations['do_validate']:
            with torch.set_grad_enabled(False):
                vpbar = tqdm(testing_generator, leave=False)
                vpbar.set_description("Validating")
                for i, (x, y) in enumerate(vpbar):
                    x = to_device(x, pyt_device)
                    y = to_device(y, pyt_device)
                    prediction = net(x)
                    loss = loss_func(prediction, y)
                    running_vloss += loss.item()
                    
                    # calculate precision https://stackoverflow.com/questions/56643503/efficient-metrics-evaluation-in-pytorch
                    predicted_classes = torch.round(torch.sigmoid(prediction)).squeeze() #.reshape(prediction.shape[0])
                    target_classes = y.squeeze() #.reshape(y.shape[0])
                    target_true += torch.sum(target_classes == 1).float()
                    predicted_true += torch.sum(predicted_classes).float()
                    correct_true += torch.sum((predicted_classes == target_classes) * (target_classes == 1)).float()
                    
            mean_vlosses = running_vloss / len(testing_generator)
            recall = 0 if target_true==0 else float((correct_true / target_true).detach())
            precision = 0 if predicted_true==0 else float((correct_true / predicted_true).detach())
            f1_score = 0 if (precision + recall)==0 else 2 * precision * recall / (precision + recall)

        mean_elosses = running_eloss / len(training_generator)
        mean_losses.append((mean_elosses, mean_vlosses, precision, recall, f1_score))
        save_model_with_config(configurations, net=net, loss_func=loss_func, optimizer=optimizer,
                               scaler=scaler,
                               mean_losses=mean_losses, next_epoch=epoch+1,)
        net.train()

        epbar.set_postfix({'train_loss':f"{mean_elosses:.6f}", 'val_loss':f"{mean_vlosses:.6f}", 'val_prec':f"{precision:.6f}"})
    net.eval()
    
    torch.cuda.empty_cache()
    return net, mean_losses

# Load and Clean Data

In [6]:
def train_test_split(X, y, train_idx=None, test_idx=None):
    X_train = X.loc[train_idx]
    y_train = y.loc[train_idx]
    X_test = X.loc[test_idx]
    y_test = y.loc[test_idx]
    return (X_train, y_train, X_test, y_test)

def load_split_data(suffix=None, split=False):
    if suffix==None:
        suffix='DEFAULT'
        
    X = pd.read_pickle(f'../data/X_{suffix}.pkl')
    y = pd.read_pickle(f'../data/y_{suffix}.pkl')
    
    # Drop NA rows:
    na_rows = X.isna().any(axis=1)
    X = X[~na_rows]
    y = y[~na_rows]
        
    if split:
        X_train, y_train, X_test, y_test = train_test_split(X, y, X.loc['2018':'2020'].index, X.loc['2021':].index)
        return X_train, y_train, X_test, y_test
    else:
        return X, y
    
X_train, y_train, X_test, y_test = load_split_data(suffix='20210806i', split=True)

In [7]:
# # Balance data...
# add_buys = (y_train.buy==0).sum() - (y_train.buy==1).sum()

# y_toadd = y_train[y_train.buy==1].sample(n=add_buys, replace=True, random_state=42)
# x_toadd = X_train[y_train.buy==1].sample(n=add_buys, replace=True, random_state=42)

# X_train = pd.concat([X_train,x_toadd])
# y_train = pd.concat([y_train,y_toadd])

#  Model Creation

In [8]:
model_id = 'nm_torch1_alpha37'

config = {
    'model_identifier' : model_id,
    'model_path' : './models',
    'model_type' : 'ResNet28',
    'data' : '20210806i',
    'note' : 're-enabled scaler',
    'device' : 'cuda',
    'random_seed' : 0,
    'max_epochs' : 20,
    'do_validate' : True,
    'model_width' : 32,
    'loss' : {
        'pos_weight' : 1
    },
    'optimizer' : {
        'type' : 'AdamW',
        'lr' : 0.006,
        'weight_decay' : 0.09, #0.00025
        'momentum' : 0.9,
    },
    'train_params' : {
        'batch_size': 1000,
        'shuffle': True,
        'num_workers': 5,
        'pin_memory': True,
    },
    'test_params' : {
        'batch_size': 200000,
        'num_workers': 5,
        'pin_memory': True,
    },
}


model_id=config['model_identifier']
with open(f'models/{model_id}.cfg', 'w') as f:
    json.dump(config, f)

# Train

In [9]:
net, mean_losses = train_model(X_train, y_train, X_test, y_test, config, force_train=False)

New model created
ResNet28(
  (stack1): ResNetStack(
    (stack): Sequential(
      (dense_1): ResnetDenseBlock(
        (dense1): Linear(in_features=542, out_features=32, bias=True)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (dense2): Linear(in_features=32, out_features=32, bias=True)
        (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dense3): Linear(in_features=32, out_features=32, bias=True)
        (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (dense_sc): Linear(in_features=542, out_features=32, bias=True)
          (bn_sc): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity_1a): ResnetIdentityBlock(
        (dense1): Linear(in_features=32, out_features=32, bias=True)
        (bn1): BatchNorm1d(32, eps=1e

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
torch.cuda.empty_cache()